In [2]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.utils import shuffle
import gc

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)

In [3]:
X_Train = pd.read_pickle(ENV.application_train_cleaned.value)
print('Train shape: {}'.format(X_Train.shape))

X_Test = pd.read_pickle(ENV.application_test_cleaned.value)
print('Test shape: {}'.format(X_Test.shape))

# X_pre = pd.read_pickle(ENV.previous_application_cleaned.value)
X_pre = pd.read_pickle(ENV.previous_application_cleaned_onehot.value)
print('Previous App shape: {}'.format(X_pre.shape))

X_bu_b = pd.read_pickle(ENV.bureau_balance_clean.value)
print('Bureau Balance shape: {}'.format(X_bu_b.shape))

# X_bu = pd.read_pickle(ENV.bureau_cleaned.value)
X_bu = pd.read_pickle(ENV.bureau_cleaned_rnnALL.value)
print('Bureau shape: {}'.format(X_bu.shape))

X_ins = pd.read_pickle(ENV.installments_payments_clean.value)
print('Installment shape: {}'.format(X_ins.shape))

X_pos = pd.read_pickle(ENV.POS_CASH_balance_clean.value)
print('POS CASH shape: {}'.format(X_pos.shape))

X_cc = pd.read_pickle(ENV.credit_card_balance_clean.value)
print('Credit Card shape: {}'.format(X_cc.shape))

Train shape: (307511, 122)
Test shape: (48744, 121)
Previous App shape: (1670214, 312)
Bureau Balance shape: (27299925, 3)
Bureau shape: (1716428, 124)
Installment shape: (13605401, 8)
POS CASH shape: (10001358, 8)
Credit Card shape: (3840312, 23)


In [8]:
def get_embeddings_index(sorted_df,nor_ebd):
    embeddings_index={}
    words_values = sorted_df['words'].values
    for index in range(len(words_values)):
        embeddings_index  [words_values[index]] = nor_ebd[index,:]
    return embeddings_index

def create_document(sorted_df):
    #Create document
    ids = sorted_df.SK_ID_CURR.values
    words = sorted_df.words.values
    document_dicts = {}

    id_list = []
    document_list = []

    for index in range(len(ids)) :
        if document_dicts.get(ids[index]) is None:
            document_dicts[ids[index]] = []
        document_dicts[ids[index]].append(words[index])

    for key in document_dicts :
        document_dicts[key] = ' '.join(document_dicts[key])
        id_list.append(key)
        document_list.append(document_dicts[key])


    df_doc = pd.DataFrame({'SK_ID_CURR':id_list, 'text':document_list})  
    df_doc_mapping  = df_doc.set_index('SK_ID_CURR').text

    train = X_Train[['SK_ID_CURR','TARGET']].copy()
    test = X_Test[['SK_ID_CURR']].copy()
    train['text'] = train.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    test['text'] = test.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    return train,test


def get_train_ebdMat(train,test,embeddings_index):
    X_train = train["text"].str.lower()
    X_test = test["text"].str.lower()
    y_train = train["TARGET"].values
    tok=text.Tokenizer(num_words=max_features,lower=True,filters='')
    tok.fit_on_texts(list(X_train)+list(X_test))
    X_train=tok.texts_to_sequences(X_train)
    X_test=tok.texts_to_sequences(X_test)
    x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
    x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
    print('...get ebd mat')
    word_index = tok.word_index
    #prepare embedding matrix
    num_words = min(max_features, len(word_index) + 1)
    print('num of words: {}'.format(num_words))
    embedding_matrix = np.zeros((num_words, embed_size))
    print(embedding_matrix.shape)
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return x_train,x_test,y_train,embedding_matrix,num_words

class_ratio =  sum(X_Train.TARGET ==0)/sum(X_Train.TARGET ==1)
class_ratio =  1
def get_rnn_model(num_words,embed_size,embedding_matrix):
    sequence_input = Input(shape=(maxlen, ))
    
    x = Embedding(num_words, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
    x = BatchNormalization()(x)
    x = SpatialDropout1D(0.2)(x)
#     x = Bidirectional(GRU(16, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Bidirectional(CuDNNGRU(8, return_sequences=True))(x)
    x = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    x = BatchNormalization()(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
#     x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
#     Adam,RMSprop,Adagrad,Adadelta,Adamax,Nadam
#     model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4),metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    return model

def train_each_epoch(x,y,batch_size,model):
    x,y = shuffle(x,y)
    model.fit(x, y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              class_weight={0:1,1:class_ratio})
    return model

def load_model(model,filepath):
    model.load_weights(filepath)
    return model

def save_model(model, filepath):
    model.save_weights(filepath)

def train_each_fold(x,y,x_val,y_val,model,filepath,reportpath,predspath,
                    batch_size=512,total_epoch=40,patience=5,saving=True):
    ROC_AUC_SCORE = []
    for epoch in range(total_epoch):  
        model = train_each_epoch(x,y,batch_size,model)
        y_pred = model.predict(x_val,batch_size=3000,verbose=1)
        score = roc_auc_score(y_val,y_pred)
        if len(ROC_AUC_SCORE) == 0:
            if saving:
                save_model(model,filepath)
            best_score = 0 
            if saving:
                print('saving preds...')
                pickle.dump(y_pred,open(predspath,'wb'))
        else:
            best_score = max(ROC_AUC_SCORE)
            if score >= best_score:
                if saving:
                    print('saving model to... {}'.format(filepath))
                    save_model(model,filepath)
                    print('saving preds...')
                    pickle.dump(y_pred,open(predspath,'wb'))
        ROC_AUC_SCORE.append(score)
        if saving:
            print('saving report to... {}'.format(reportpath))
            pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
        print('======= current {} / {}'.format(epoch,total_epoch))
        print('previous best roc is {}'.format(best_score))
        print('current roc is {}'.format(score))
        try:
            best_round = ROC_AUC_SCORE.index(best_score)
        except ValueError:
            best_round = -1
        if len(ROC_AUC_SCORE) > patience + best_round:
            print('reach patience! end loop')
            break
            
def train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=512,total_epoch=400,patience=30):
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(0,len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra = x_train[train_index]
        y_tra = y_train[train_index]
        X_val = x_train[val_index]
        y_val = y_train[val_index]
        print('preparing train/val done!')
        print('before evaluating: {}'.format(model_file))
        model_file_evl = model_file.format(fold)
        report_file_evl = report_file.format(fold)
        pred_file_evl = pred_file.format(fold)
        pred_test_file_evl = pred_test_file.format(fold)
        model = get_rnn_model(num_words,embed_size,embedding_matrix)
        train_each_fold(X_tra, y_tra, X_val, y_val,
                        model,
                        filepath=model_file_evl,reportpath=report_file_evl,predspath=pred_file_evl,
                        batch_size=batch_size,total_epoch=total_epoch,patience=patience)
        gc.collect()
        #### predict test
        model = load_model(model,model_file_evl)
        test_preds = model.predict(x_test,batch_size=1500,verbose=1)
        pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
        print('\n')


# Previous application

In [5]:
# qt = QuantileTransformer(n_quantiles=100000,output_distribution='normal')

In [ ]:
# qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
# trans_col = ['AMT_ANNUITY',
#              'AMT_APPLICATION',
#              'AMT_CREDIT',
#              'AMT_DOWN_PAYMENT',
#              'AMT_GOODS_PRICE',
#              'RATE_DOWN_PAYMENT',
#              'RATE_INTEREST_PRIMARY',
#              'RATE_INTEREST_PRIVILEGED',
#              'SELLERPLACE_AREA',
#              'CNT_PAYMENT',
#              'DAYS_FIRST_DRAWING',
#              'DAYS_FIRST_DUE',
#              'DAYS_LAST_DUE_1ST_VERSION',
#              'DAYS_LAST_DUE',
#              'DAYS_TERMINATION']
# for col in trans_col:
#     print(col)
#     X_pre[col] = qt.fit_transform(X_pre[col].values.reshape(-1,1))

In [6]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = ['AMT_ANNUITY',
 'AMT_APPLICATION',
 'AMT_CREDIT',
 'AMT_DOWN_PAYMENT',
 'AMT_GOODS_PRICE',
 'RATE_DOWN_PAYMENT',
 'RATE_INTEREST_PRIMARY',
 'RATE_INTEREST_PRIVILEGED',
 'SELLERPLACE_AREA',
 'CNT_PAYMENT',
 'DAYS_FIRST_DRAWING',
 'DAYS_FIRST_DUE',
 'DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_LAST_DUE',
 'DAYS_TERMINATION',
 'AMT_ANNUITY_squre',
 'AMT_APPLICATION_squre',
 'AMT_CREDIT_squre',
 'AMT_DOWN_PAYMENT_squre',
 'AMT_GOODS_PRICE_squre',
 'RATE_DOWN_PAYMENT_squre',
 'RATE_INTEREST_PRIMARY_squre',
 'RATE_INTEREST_PRIVILEGED_squre',
 'SELLERPLACE_AREA_squre',
 'CNT_PAYMENT_squre',
 'DAYS_FIRST_DRAWING_squre',
 'DAYS_FIRST_DUE_squre',
 'DAYS_LAST_DUE_1ST_VERSION_squre',
 'DAYS_LAST_DUE_squre',
 'DAYS_TERMINATION_squre',
 'AMT_ANNUITY_AMT_APPLICATION',
 'AMT_ANNUITY_AMT_CREDIT',
 'AMT_ANNUITY_AMT_DOWN_PAYMENT',
 'AMT_ANNUITY_AMT_GOODS_PRICE',
 'AMT_ANNUITY_RATE_DOWN_PAYMENT',
 'AMT_ANNUITY_RATE_INTEREST_PRIMARY',
 'AMT_ANNUITY_RATE_INTEREST_PRIVILEGED',
 'AMT_ANNUITY_SELLERPLACE_AREA',
 'AMT_ANNUITY_CNT_PAYMENT',
 'AMT_ANNUITY_DAYS_FIRST_DRAWING',
 'AMT_ANNUITY_DAYS_FIRST_DUE',
 'AMT_ANNUITY_DAYS_LAST_DUE_1ST_VERSION',
 'AMT_ANNUITY_DAYS_LAST_DUE',
 'AMT_ANNUITY_DAYS_TERMINATION',
 'AMT_APPLICATION_AMT_CREDIT',
 'AMT_APPLICATION_AMT_DOWN_PAYMENT',
 'AMT_APPLICATION_AMT_GOODS_PRICE',
 'AMT_APPLICATION_RATE_DOWN_PAYMENT',
 'AMT_APPLICATION_RATE_INTEREST_PRIMARY',
 'AMT_APPLICATION_RATE_INTEREST_PRIVILEGED',
 'AMT_APPLICATION_SELLERPLACE_AREA',
 'AMT_APPLICATION_CNT_PAYMENT',
 'AMT_APPLICATION_DAYS_FIRST_DRAWING',
 'AMT_APPLICATION_DAYS_FIRST_DUE',
 'AMT_APPLICATION_DAYS_LAST_DUE_1ST_VERSION',
 'AMT_APPLICATION_DAYS_LAST_DUE',
 'AMT_APPLICATION_DAYS_TERMINATION',
 'AMT_CREDIT_AMT_DOWN_PAYMENT',
 'AMT_CREDIT_AMT_GOODS_PRICE',
 'AMT_CREDIT_RATE_DOWN_PAYMENT',
 'AMT_CREDIT_RATE_INTEREST_PRIMARY',
 'AMT_CREDIT_RATE_INTEREST_PRIVILEGED',
 'AMT_CREDIT_SELLERPLACE_AREA',
 'AMT_CREDIT_CNT_PAYMENT',
 'AMT_CREDIT_DAYS_FIRST_DRAWING',
 'AMT_CREDIT_DAYS_FIRST_DUE',
 'AMT_CREDIT_DAYS_LAST_DUE_1ST_VERSION',
 'AMT_CREDIT_DAYS_LAST_DUE',
 'AMT_CREDIT_DAYS_TERMINATION',
 'AMT_DOWN_PAYMENT_AMT_GOODS_PRICE',
 'AMT_DOWN_PAYMENT_RATE_DOWN_PAYMENT',
 'AMT_DOWN_PAYMENT_RATE_INTEREST_PRIMARY',
 'AMT_DOWN_PAYMENT_RATE_INTEREST_PRIVILEGED',
 'AMT_DOWN_PAYMENT_SELLERPLACE_AREA',
 'AMT_DOWN_PAYMENT_CNT_PAYMENT',
 'AMT_DOWN_PAYMENT_DAYS_FIRST_DRAWING',
 'AMT_DOWN_PAYMENT_DAYS_FIRST_DUE',
 'AMT_DOWN_PAYMENT_DAYS_LAST_DUE_1ST_VERSION',
 'AMT_DOWN_PAYMENT_DAYS_LAST_DUE',
 'AMT_DOWN_PAYMENT_DAYS_TERMINATION',
 'AMT_GOODS_PRICE_RATE_DOWN_PAYMENT',
 'AMT_GOODS_PRICE_RATE_INTEREST_PRIMARY',
 'AMT_GOODS_PRICE_RATE_INTEREST_PRIVILEGED',
 'AMT_GOODS_PRICE_SELLERPLACE_AREA',
 'AMT_GOODS_PRICE_CNT_PAYMENT',
 'AMT_GOODS_PRICE_DAYS_FIRST_DRAWING',
 'AMT_GOODS_PRICE_DAYS_FIRST_DUE',
 'AMT_GOODS_PRICE_DAYS_LAST_DUE_1ST_VERSION',
 'AMT_GOODS_PRICE_DAYS_LAST_DUE',
 'AMT_GOODS_PRICE_DAYS_TERMINATION',
 'RATE_DOWN_PAYMENT_RATE_INTEREST_PRIMARY',
 'RATE_DOWN_PAYMENT_RATE_INTEREST_PRIVILEGED',
 'RATE_DOWN_PAYMENT_SELLERPLACE_AREA',
 'RATE_DOWN_PAYMENT_CNT_PAYMENT',
 'RATE_DOWN_PAYMENT_DAYS_FIRST_DRAWING',
 'RATE_DOWN_PAYMENT_DAYS_FIRST_DUE',
 'RATE_DOWN_PAYMENT_DAYS_LAST_DUE_1ST_VERSION',
 'RATE_DOWN_PAYMENT_DAYS_LAST_DUE',
 'RATE_DOWN_PAYMENT_DAYS_TERMINATION',
 'RATE_INTEREST_PRIMARY_RATE_INTEREST_PRIVILEGED',
 'RATE_INTEREST_PRIMARY_SELLERPLACE_AREA',
 'RATE_INTEREST_PRIMARY_CNT_PAYMENT',
 'RATE_INTEREST_PRIMARY_DAYS_FIRST_DRAWING',
 'RATE_INTEREST_PRIMARY_DAYS_FIRST_DUE',
 'RATE_INTEREST_PRIMARY_DAYS_LAST_DUE_1ST_VERSION',
 'RATE_INTEREST_PRIMARY_DAYS_LAST_DUE',
 'RATE_INTEREST_PRIMARY_DAYS_TERMINATION',
 'RATE_INTEREST_PRIVILEGED_SELLERPLACE_AREA',
 'RATE_INTEREST_PRIVILEGED_CNT_PAYMENT',
 'RATE_INTEREST_PRIVILEGED_DAYS_FIRST_DRAWING',
 'RATE_INTEREST_PRIVILEGED_DAYS_FIRST_DUE',
 'RATE_INTEREST_PRIVILEGED_DAYS_LAST_DUE_1ST_VERSION',
 'RATE_INTEREST_PRIVILEGED_DAYS_LAST_DUE',
 'RATE_INTEREST_PRIVILEGED_DAYS_TERMINATION',
 'SELLERPLACE_AREA_CNT_PAYMENT',
 'SELLERPLACE_AREA_DAYS_FIRST_DRAWING',
 'SELLERPLACE_AREA_DAYS_FIRST_DUE',
 'SELLERPLACE_AREA_DAYS_LAST_DUE_1ST_VERSION',
 'SELLERPLACE_AREA_DAYS_LAST_DUE',
 'SELLERPLACE_AREA_DAYS_TERMINATION',
 'CNT_PAYMENT_DAYS_FIRST_DRAWING',
 'CNT_PAYMENT_DAYS_FIRST_DUE',
 'CNT_PAYMENT_DAYS_LAST_DUE_1ST_VERSION',
 'CNT_PAYMENT_DAYS_LAST_DUE',
 'CNT_PAYMENT_DAYS_TERMINATION',
 'DAYS_FIRST_DRAWING_DAYS_FIRST_DUE',
 'DAYS_FIRST_DRAWING_DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_FIRST_DRAWING_DAYS_LAST_DUE',
 'DAYS_FIRST_DRAWING_DAYS_TERMINATION',
 'DAYS_FIRST_DUE_DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_FIRST_DUE_DAYS_LAST_DUE',
 'DAYS_FIRST_DUE_DAYS_TERMINATION',
 'DAYS_LAST_DUE_1ST_VERSION_DAYS_LAST_DUE',
 'DAYS_LAST_DUE_1ST_VERSION_DAYS_TERMINATION',
 'DAYS_LAST_DUE_DAYS_TERMINATION']
for col in trans_col:
    print(col)
    X_pre[col] = qt.fit_transform(X_pre[col].values.reshape(-1,1))

AMT_ANNUITY
AMT_APPLICATION
AMT_CREDIT
AMT_DOWN_PAYMENT
AMT_GOODS_PRICE
RATE_DOWN_PAYMENT
RATE_INTEREST_PRIMARY
RATE_INTEREST_PRIVILEGED
SELLERPLACE_AREA
CNT_PAYMENT
DAYS_FIRST_DRAWING
DAYS_FIRST_DUE
DAYS_LAST_DUE_1ST_VERSION
DAYS_LAST_DUE
DAYS_TERMINATION
AMT_ANNUITY_squre
AMT_APPLICATION_squre
AMT_CREDIT_squre
AMT_DOWN_PAYMENT_squre
AMT_GOODS_PRICE_squre
RATE_DOWN_PAYMENT_squre
RATE_INTEREST_PRIMARY_squre
RATE_INTEREST_PRIVILEGED_squre
SELLERPLACE_AREA_squre
CNT_PAYMENT_squre
DAYS_FIRST_DRAWING_squre
DAYS_FIRST_DUE_squre
DAYS_LAST_DUE_1ST_VERSION_squre
DAYS_LAST_DUE_squre
DAYS_TERMINATION_squre
AMT_ANNUITY_AMT_APPLICATION
AMT_ANNUITY_AMT_CREDIT
AMT_ANNUITY_AMT_DOWN_PAYMENT
AMT_ANNUITY_AMT_GOODS_PRICE
AMT_ANNUITY_RATE_DOWN_PAYMENT
AMT_ANNUITY_RATE_INTEREST_PRIMARY
AMT_ANNUITY_RATE_INTEREST_PRIVILEGED
AMT_ANNUITY_SELLERPLACE_AREA
AMT_ANNUITY_CNT_PAYMENT
AMT_ANNUITY_DAYS_FIRST_DRAWING
AMT_ANNUITY_DAYS_FIRST_DUE
AMT_ANNUITY_DAYS_LAST_DUE_1ST_VERSION
AMT_ANNUITY_DAYS_LAST_DUE
AMT_ANNUITY_

In [9]:
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
max_features = 1680000
# max 73
maxlen = 12
# 35
# embed_size = 35

sorted_df = X_pre.sort_values(['SK_ID_CURR','DAYS_DECISION']).copy()
col = 'DAYS_DECISION'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
###################################################
# feature.remove('DAYS_FIRST_DRAWING')
# feature.remove('DAYS_FIRST_DUE')
# feature.remove('DAYS_LAST_DUE_1ST_VERSION')
# feature.remove('DAYS_LAST_DUE')
# feature.remove('DAYS_TERMINATION')
###################################################
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.previous_application_rnn.value
report_file = ENV.previous_application_report.value
pred_file = ENV.previous_application_preds.value
pred_test_file = ENV.previous_application_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=800,patience=45)

ebd size is 310
start normalize
get embedding
create document
get embedding Mat
...get ebd mat
num of words: 1670216
(1670216, 310)
!!!!!!!! Begin fold: 3
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/previous_application/fold_{}.hdf5
Epoch 1/1
61502/61502 [==============================] - 0s 4us/step
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 0 / 800
previous best roc is 0
current roc is 0.6101735075063452
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_3.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 1 / 800
previous best roc is 0.6101735075063452
current roc is 0.6352473804964205
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_3.hdf5
saving preds...


Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 21 / 800
previous best roc is 0.6702355379183171
current roc is 0.6687662639955564
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_3.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 22 / 800
previous best roc is 0.6702355379183171
current roc is 0.6706845759799855
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 23 / 800
previous best roc is 0.6706845759799855
current roc is 0.6699398704951728
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= curre

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 45 / 800
previous best roc is 0.6706845759799855
current roc is 0.6633873091056031
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 46 / 800
previous best roc is 0.6706845759799855
current roc is 0.66439684531416
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 47 / 800
previous best roc is 0.6706845759799855
current roc is 0.6632517533169326
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 48 / 800
previous best roc is 0.6706845759799855
current roc is 0.6640897710157978
Epoch 

61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 1 / 800
previous best roc is 0.61786574748442
current roc is 0.6395294423836584
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 2 / 800
previous best roc is 0.6395294423836584
current roc is 0.6505840349693011
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 3 / 800
previous best roc is 0.6505840349693011
current roc is 0.658203024137885

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 22 / 800
previous best roc is 0.6693242225603842
current roc is 0.667371548303849
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 23 / 800
previous best roc is 0.6693242225603842
current roc is 0.6697521826014532
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 24 / 800
previous best roc is 0.6697521826014532
current roc is 0.6672547240961879
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= curren

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 46 / 800
previous best roc is 0.6715576589711141
current roc is 0.6686636827338548
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 47 / 800
previous best roc is 0.6715576589711141
current roc is 0.6656220312522194
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 48 / 800
previous best roc is 0.6715576589711141
current roc is 0.6675124537509292
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 49 / 800
previous best roc is 0.6715576589711141
current roc is 0.6675883210465853
Epoc

In [ ]:
aaa

# Bureau

In [ ]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = [
             'AMT_ANNUITY',
             'AMT_CREDIT_MAX_OVERDUE',
             'DAYS_ENDDATE_FACT',
             'AMT_CREDIT_SUM_LIMIT',
             'AMT_CREDIT_SUM_DEBT',
             'DAYS_CREDIT_ENDDATE',
             'AMT_CREDIT_SUM',
             'CREDIT_DAY_OVERDUE',
             'CNT_CREDIT_PROLONG',
             'BUREAU_LENGTH',
             'AMT_CREDIT_SUM_OVERDUE',
             'DAYS_CREDIT_UPDATE']

trans_col2 = ['AMT_ANNUITY_squre',
 'AMT_CREDIT_MAX_OVERDUE_squre',
 'DAYS_ENDDATE_FACT_squre',
 'AMT_CREDIT_SUM_LIMIT_squre',
 'AMT_CREDIT_SUM_DEBT_squre',
 'DAYS_CREDIT_ENDDATE_squre',
 'AMT_CREDIT_SUM_squre',
 'CREDIT_DAY_OVERDUE_squre',
 'CNT_CREDIT_PROLONG_squre',
 'BUREAU_LENGTH_squre',
 'AMT_CREDIT_SUM_OVERDUE_squre',
 'DAYS_CREDIT_UPDATE_squre']

trans_col3 = ['AMT_ANNUITY_AMT_CREDIT_MAX_OVERDUE',
 'AMT_ANNUITY_DAYS_ENDDATE_FACT',
 'AMT_ANNUITY_AMT_CREDIT_SUM_LIMIT',
 'AMT_ANNUITY_AMT_CREDIT_SUM_DEBT',
 'AMT_ANNUITY_DAYS_CREDIT_ENDDATE',
 'AMT_ANNUITY_AMT_CREDIT_SUM',
 'AMT_ANNUITY_CREDIT_DAY_OVERDUE',
 'AMT_ANNUITY_CNT_CREDIT_PROLONG',
 'AMT_ANNUITY_BUREAU_LENGTH',
 'AMT_ANNUITY_AMT_CREDIT_SUM_OVERDUE',
 'AMT_ANNUITY_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM',
 'AMT_CREDIT_MAX_OVERDUE_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_MAX_OVERDUE_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_MAX_OVERDUE_BUREAU_LENGTH',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_CREDIT_UPDATE',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_LIMIT',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_DEBT',
 'DAYS_ENDDATE_FACT_DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM',
 'DAYS_ENDDATE_FACT_CREDIT_DAY_OVERDUE',
 'DAYS_ENDDATE_FACT_CNT_CREDIT_PROLONG',
 'DAYS_ENDDATE_FACT_BUREAU_LENGTH',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_OVERDUE',
 'DAYS_ENDDATE_FACT_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_LIMIT_AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_SUM_LIMIT_AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_LIMIT_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_SUM_LIMIT_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM_LIMIT_BUREAU_LENGTH',
 'AMT_CREDIT_SUM_LIMIT_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_SUM_LIMIT_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_DEBT_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_SUM_DEBT_AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_SUM_DEBT_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM_DEBT_BUREAU_LENGTH',
 'AMT_CREDIT_SUM_DEBT_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_SUM_DEBT_DAYS_CREDIT_UPDATE',
 'DAYS_CREDIT_ENDDATE_AMT_CREDIT_SUM',
 'DAYS_CREDIT_ENDDATE_CREDIT_DAY_OVERDUE',
 'DAYS_CREDIT_ENDDATE_CNT_CREDIT_PROLONG',
 'DAYS_CREDIT_ENDDATE_BUREAU_LENGTH',
 'DAYS_CREDIT_ENDDATE_AMT_CREDIT_SUM_OVERDUE',
 'DAYS_CREDIT_ENDDATE_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_SUM_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM_BUREAU_LENGTH',
 'AMT_CREDIT_SUM_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_SUM_DAYS_CREDIT_UPDATE',
 'CREDIT_DAY_OVERDUE_CNT_CREDIT_PROLONG',
 'CREDIT_DAY_OVERDUE_BUREAU_LENGTH',
 'CREDIT_DAY_OVERDUE_AMT_CREDIT_SUM_OVERDUE',
 'CREDIT_DAY_OVERDUE_DAYS_CREDIT_UPDATE',
 'CNT_CREDIT_PROLONG_BUREAU_LENGTH',
 'CNT_CREDIT_PROLONG_AMT_CREDIT_SUM_OVERDUE',
 'CNT_CREDIT_PROLONG_DAYS_CREDIT_UPDATE',
 'BUREAU_LENGTH_AMT_CREDIT_SUM_OVERDUE',
 'BUREAU_LENGTH_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_SUM_OVERDUE_DAYS_CREDIT_UPDATE']
for col in trans_col + trans_col2 + trans_col3:
    print(col)
    X_bu[col] = qt.fit_transform(X_bu[col].values.reshape(-1,1))

In [ ]:
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#1760000
max_features = 1720000
#116
maxlen = 13
# embed_size = 15

sorted_df = X_bu.sort_values(['SK_ID_CURR','DAYS_CREDIT']).copy()
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
col = 'DAYS_CREDIT'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
feature.remove('SK_ID_BUREAU')
feature.remove('SK_ID_CURR')
feature.remove('words')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.bureau_rnn.value
report_file = ENV.bureau_report.value
pred_file = ENV.bureau_preds.value
pred_test_file = ENV.bureau_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=2000,patience=100)


In [ ]:
max(pickle.load(open(ENV.bureau_report.value.format(3),'rb')))

In [ ]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

# Installment

In [ ]:


label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#13605401
max_features = 13605402
#372
maxlen = 60
embed_size = 6

sorted_df = X_ins.sort_values(['SK_ID_CURR','DAYS_INSTALMENT']).copy()
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.installment_rnn.value
report_file = ENV.installment_report.value
pred_file = ENV.installment_preds.value
pred_test_file = ENV.installment_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=30)

In [ ]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

In [4]:
import pickle
max(pickle.load(open(ENV.installment_report.value.format(1),'rb')))

0.6493842331083122

# POS_CASH

In [ ]:

print('start')
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 10001359
#295
maxlen = 50
embed_size = 6

sorted_df = X_pos.sort_values(['SK_ID_CURR','MONTHS_BALANCE'])
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.POS_CASH_rnn.value
report_file = ENV.POS_CASH_report.value
pred_file = ENV.POS_CASH_preds.value
pred_test_file = ENV.POS_CASH_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=30)

In [ ]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

# Credit Card

In [ ]:


label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 10001359
#192
maxlen = 15
embed_size = 21

sorted_df = X_cc.sort_values(['SK_ID_CURR','MONTHS_BALANCE'])
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.credit_card_rnn.value
report_file = ENV.credit_card_report.value
pred_file = ENV.credit_card_preds.value
pred_test_file = ENV.credit_card_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=30)

In [ ]:
len(feature)

In [ ]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

In [ ]:
len(sorted_df['words'].unique())

In [ ]:
len(feature)

In [ ]:
max(train.text.apply(lambda x:len(x.split(' '))))

In [ ]:
max(test.text.apply(lambda x:len(x.split(' '))))

In [ ]:
train

# sort by sk curr and decision day

In [ ]:
sorted_pre = X_pre.sort_values(['SK_ID_CURR','DAYS_DECISION'])

# Use SK_ID_PREV as words

In [ ]:
sorted_pre['words'] = sorted_pre.SK_ID_PREV.astype(str)

# Get feature cols

In [ ]:
feature = list(sorted_pre.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')

# Get EBD

In [ ]:
ebd = sorted_pre[feature].values

# Nomralize

In [ ]:
nor_ebd = normalize(ebd, norm='max',axis=0)

# Get word EBD

In [ ]:
embeddings_index   = {}
words_values = sorted_pre['words'].values
for index in range(len(words_values)):
    embeddings_index  [words_values[index]] = nor_ebd[index,:]

# Train 5 folds

In [ ]:
def train_5_folds(model_file,report_file,pred_file,pred_test_file):
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra = x_train[train_index]
        y_tra = y_train[train_index]
        X_val = x_train[val_index]
        y_val = y_train[val_index]
        print('preparing train/val done!')
        model = get_rnn_model(num_words,embed_size,embedding_matrix)
        train_each_fold(X_tra, y_tra, X_val, y_val,
                        model,
                        filepath=model_file,reportpath=report_file,predspath=pred_file,
                        batch_size=512,total_epoch=40)
        #### predict test
        model = load_model(model,model_file)
        test_preds = model.predict(x_test,batch_size=5120,verbose=1)
        pickle.dump(test_preds,open(pred_test_file,'wb'))
        print('\n')

In [ ]:
train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

for fold in range(len(train_fold_index)):
    print('!!!!!!!! Begin fold: {}'.format(fold))
    train_index = train_fold_index[fold]
    val_index = val_fold_index[fold]
    X_tra = x_train[train_index]
    y_tra = y_train[train_index]
    X_val = x_train[val_index]
    y_val = y_train[val_index]
    print('preparing train/val done!')
    model_file = ENV.previous_application_rnn.value.format(fold)
    report_file = ENV.previous_application_report.value.format(fold)
    pred_file = ENV.previous_application_preds.value.format(fold)
    pred_test_file = ENV.previous_application_preds_test.value.format(fold)
    model = get_rnn_model(num_words,embed_size,embedding_matrix)
    train_each_fold(X_tra, y_tra, X_val, y_val,
                    model,
                    filepath=model_file,reportpath=report_file,predspath=pred_file,
                    batch_size=512,total_epoch=40)
    #### predict test
    model = load_model(model,model_file)
    test_preds = model.predict(x_test,batch_size=5120,verbose=1)
    pickle.dump(test_preds,open(pred_test_file,'wb'))
    print('\n')

In [ ]:
ssss

In [ ]:
sequence_input = Input(shape=(maxlen, ))
x = Embedding(num_words, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 1, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)
preds = Dense(1, activation="sigmoid")(x)
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

In [ ]:
batch_size = 512
epochs = 40
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state=233)

In [ ]:
# filepath="../input/best-model/best.hdf5"
filepath="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_loss", mode="max", patience=40)
ra_val = RocAucEvaluation(validation_data=(X_val, y_val), interval = 1)
callbacks_list = [ra_val,checkpoint, early]

## Train

In [ ]:
model.save_weights("weights_base.best123.hdf5")

In [ ]:
model.load_weights("weights_base.best123.hdf5")

In [ ]:

preds = model.predict(X_val,batch_size=5120,verbose=1)

In [ ]:
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1,class_weight={0:1,1:11.5})

In [ ]:
x = list(range(1000))
y = list(range(1000))

In [ ]:
x1,y1 = shuffle(x,y)
print(x1)
print(y1)

In [ ]:
x1.copy()